In [1]:

import dataretrieval.nwis as nwis
import geopandas as gpd
from shapely.geometry import Point, box, Polygon, MultiPolygon
import requests
import pandas as pd
import datetime as dt
import earthaccess
from tqdm import tqdm
from shapely.ops import nearest_points
import sys
sys.path.append('modules/')
from retrieval_utils_appeearsTest import get_param_sites, get_scenes_results, get_scenes_temp, get_scenes_turb
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import numpy as np
import geopy
import ast
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm


In [104]:
# import results, ensure spectra is list of dicts
param_codes = ['32316']
results = pd.read_csv(f"data/{param_codes[0]}_results.csv", dtype={'site_no': str})
results['spectra'] = results['spectra'].apply(ast.literal_eval)

# query the USGS database for temperature results, merge tables
temps = get_scenes_temp(results)
results['datetime'] = pd.to_datetime(results['datetime'], utc=True)
temps['datetime'] = pd.to_datetime(temps['datetime'], utc=True)
comb_results = pd.merge(results, temps, on=['site_no', 'datetime'], how='left')


Finding temperature for scenes: 100%|███████████| 93/93 [00:45<00:00,  2.05it/s]


In [105]:
# query the USGS database for turbidity results, merge tables
comb_results = comb_results.dropna(subset=['temp'])
turbs = get_scenes_turb(comb_results)
comb_results['datetime'] = pd.to_datetime(comb_results['datetime'], utc=True)
turbs['datetime'] = pd.to_datetime(turbs['datetime'], utc=True)
turb_results = pd.merge(comb_results, turbs, on=['site_no', 'datetime'], how='left')

Finding turbidity for scenes: 100%|█████████████| 92/92 [00:43<00:00,  2.10it/s]


In [112]:
# categorise rows into clear or turbid waters based on results
turb_results['turbidity'] = pd.to_numeric(turb_results['turbidity'], errors='coerce')
results_clean = turb_results.dropna(subset=['turbidity']).copy()
def classify_water(row):
    if row['turbidity'] <= 15:
        return 'clear'
    else:
        return 'turbid'
results_clean['water_type'] = results_clean.apply(classify_water, axis=1)

# Split the dataset into two groups\
clear_water = results_clean[results_clean['water_type'] == 'clear']
turbid_water = results_clean[results_clean['water_type'] == 'turbid']
print(results_clean['water_type'])

0      clear
1     turbid
2      clear
4     turbid
5     turbid
       ...  
87     clear
88     clear
89     clear
90     clear
91     clear
Name: water_type, Length: 84, dtype: object


In [113]:
# basic descriptive statistics for turbid water
turbid_stats = turbid_water[['turbidity', 'result', 'temp']].astype(float).describe().round(2)
print(turbid_stats)


       turbidity  result   temp
count      38.00   38.00  38.00
mean       36.27    9.63  21.74
std        20.70    7.21   5.95
min        15.20    2.10  10.60
25%        21.98    3.84  19.45
50%        30.60    7.55  21.70
75%        42.08   13.94  26.67
max       118.00   29.29  32.10


In [115]:

# standardise data
turbid_water = turbid_water.copy()
data_to_standardise = turbid_water[['turbidity', 'result', 'temp']].astype(float)
scaler = StandardScaler()
standardised_data = scaler.fit_transform(data_to_standardise)
turbid_water[['turbidity_standard', 'result_standard', 'temp_standard']] = standardised_data
print(turbid_water[['turbidity_standard', 'result_standard', 'temp_standard']].head())


    turbidity_standard  result_standard  temp_standard
1             0.084885        -0.838120      -0.398732
4             2.365827        -0.582376      -0.279561
5             0.593936        -0.752404       0.878107
9            -0.448640         0.961924       0.912156
11           -0.458430         1.074339      -0.228487


In [117]:
clear_water = clear_water.copy()
turbid_water = turbid_water.copy()

# target bands
blue_range = (450, 495)
green_range = (495, 570)
red_range = (620, 750)
nir_range = (750, 900)

# filter spectra for target bands
def filter_bands(spectra, band_ranges):
    filtered_spectra = {'blue': [], 'green': [], 'red': [], 'nir': []}
    for scene in spectra:
        wavelength = scene['wavelength']
        reflectance = scene['reflectance']
        if reflectance == -9999.0:
            continue
        if blue_range[0] <= wavelength <= blue_range[1]:
            filtered_spectra['blue'].append(reflectance)
        elif green_range[0] <= wavelength <= green_range[1]:
            filtered_spectra['green'].append(reflectance)
        elif red_range[0] <= wavelength <= red_range[1]:
            filtered_spectra['red'].append(reflectance)
        elif nir_range[0] <= wavelength <= nir_range[1]:
            filtered_spectra['nir'].append(reflectance)
    
    # calculate mean reflectance for each band
    for band in filtered_spectra:
        filtered_spectra[band] = np.mean(filtered_spectra[band]) if filtered_spectra[band] else np.nan
    return pd.Series(filtered_spectra)

# Apply the filter to the 'spectra' column for both clear and turbid water datasets
clear_water[['blue_mean', 'green_mean', 'red_mean', 'nir_mean']] = clear_water['spectra'].apply(
    lambda x: filter_bands(x, [blue_range, green_range, red_range, nir_range]))
turbid_water[['blue_mean', 'green_mean', 'red_mean', 'nir_mean']] = turbid_water['spectra'].apply(
    lambda x: filter_bands(x, [blue_range, green_range, red_range, nir_range]))

# standardize the filtered reflectance values for the turbid water
reflectance_to_standardize = turbid_water[['blue_mean', 'green_mean', 'red_mean', 'nir_mean']].astype(float)
reflectance_scaler = StandardScaler()
reflectance_standardized = reflectance_scaler.fit_transform(reflectance_to_standardize)
turbid_water[['blue_standard', 'green_standard', 'red_standard', 'nir_standard']] = reflectance_standardized

# Remove rows where reflectance data is missing (optional)
turbid_water.dropna(subset=['blue_mean', 'green_mean', 'red_mean', 'nir_mean'], inplace=True)
clear_water.dropna(subset=['blue_mean', 'green_mean', 'red_mean', 'nir_mean'], inplace=True)

# Reset index
turbid_water.reset_index(drop=True, inplace=True)
clear_water.reset_index(drop=True, inplace=True)

# Display the filtered and standardized results
print(turbid_water[['blue_standard', 'green_standard', 'red_standard', 'nir_standard']].head())


   blue_standard  green_standard  red_standard  nir_standard
0       0.068895       -0.280634     -0.025308      0.173221
1      -1.144734       -1.184893     -0.498047      0.664423
2       0.021258       -0.052572      0.410800      1.134019
3       0.890539        0.509206      0.347243      0.776794
4      -0.881056       -1.003274     -0.810646     -0.717300


In [118]:
# combine all standardised columns and create statistics
standardized_data_all = turbid_water[['turbidity_standard', 'result_standard', 'temp_standard', 'blue_standard', 'green_standard', 'red_standard', 'nir_standard']]
desc_stats_all = standardized_data_all.describe()
desc_stats_all = desc_stats_all.round(2)
print(desc_stats_all.head())

       turbidity_standard  result_standard  temp_standard  blue_standard  \
count               35.00            35.00          35.00          35.00   
mean                 0.05             0.01          -0.06           0.00   
std                  1.03             1.05           1.03           1.01   
min                 -0.99            -1.06          -1.90          -2.43   
25%                 -0.68            -0.83          -0.40          -0.71   

       green_standard  red_standard  nir_standard  
count           35.00         35.00         35.00  
mean            -0.00          0.00         -0.00  
std              1.01          1.01          1.01  
min             -2.16         -1.71         -1.80  
25%             -0.80         -0.79         -0.58  


In [119]:
#X = turbid_water[['turbidity_standard', 'result_standard', 'temp_standard', 'blue_standard', 'green_standard', 'red_standard', 'nir_standard']]
X = clear_water[['blue_standard', 'green_standard','red_standard', 'nir_standard']]
y = clear_water['result_standard']
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
print(model.summary())
print(len(clear_water))

KeyError: "None of [Index(['blue_standard', 'green_standard', 'red_standard', 'nir_standard'], dtype='object')] are in the [columns]"

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
'''
# Define your dataframes
clear_water = clear_water.copy()  # Ensure no SettingWithCopyWarnings
turbid_water = turbid_water.copy()

# Create a scatter plot for each band vs. chlorophyll-a concentration
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
sns.regplot(x='blue_standard', y='result_standard', data=turbid_water, ax=axes[0, 0], color='blue')
axes[0, 0].set_title('Blue Band vs Chla RFU')
axes[0, 0].legend()

sns.regplot(x='green_standard', y='result_standard', data=turbid_water, ax=axes[0, 1], color='green')
axes[0, 1].set_title('Green Band vs Chla RFU')

sns.regplot(x='red_standard', y='result_standard', data=turbid_water, ax=axes[1, 0], color='red')
axes[1, 0].set_title('Red Band vs Chla RFU')

sns.regplot(x='nir_standard', y='result_standard', data=turbid_water, ax=axes[1, 1], color='purple')
axes[1, 1].set_title('NIR Band vs Chla RFU')

plt.tight_layout()
plt.show()
'''

"\n# Define your dataframes\nclear_water = clear_water.copy()  # Ensure no SettingWithCopyWarnings\nturbid_water = turbid_water.copy()\n\n# Create a scatter plot for each band vs. chlorophyll-a concentration\nfig, axes = plt.subplots(2, 2, figsize=(12, 10))\nsns.regplot(x='blue_standard', y='result_standard', data=turbid_water, ax=axes[0, 0], color='blue')\naxes[0, 0].set_title('Blue Band vs Chla RFU')\naxes[0, 0].legend()\n\nsns.regplot(x='green_standard', y='result_standard', data=turbid_water, ax=axes[0, 1], color='green')\naxes[0, 1].set_title('Green Band vs Chla RFU')\n\nsns.regplot(x='red_standard', y='result_standard', data=turbid_water, ax=axes[1, 0], color='red')\naxes[1, 0].set_title('Red Band vs Chla RFU')\n\nsns.regplot(x='nir_standard', y='result_standard', data=turbid_water, ax=axes[1, 1], color='purple')\naxes[1, 1].set_title('NIR Band vs Chla RFU')\n\nplt.tight_layout()\nplt.show()\n"

In [1]:

'''
# Select the features for clustering (standardized reflectance and chlorophyll-a)
features_for_clustering = turbid_water[['blue_standard', 'green_standard', 'red_standard', 'nir_standard', 'result_standard']]

# Define the number of clusters (you can adjust this based on your data exploration)
kmeans = KMeans(n_clusters=3, random_state=42)  # Start with 3 clusters
turbid_water['cluster'] = kmeans.fit_predict(features_for_clustering)

# Create a scatter plot for each band with clusters
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Blue Band
sns.scatterplot(x='blue_standard', y='result_standard', hue='cluster', data=turbid_water, palette='Set1', ax=axes[0, 0])
axes[0, 0].set_title('Blue Band vs Chla RFU')

# Green Band
sns.scatterplot(x='green_standard', y='result_standard', hue='cluster', data=turbid_water, palette='Set1', ax=axes[0, 1])
axes[0, 1].set_title('Green Band vs Chla RFU')

# Red Band
sns.scatterplot(x='red_standard', y='result_standard', hue='cluster', data=turbid_water, palette='Set1', ax=axes[1, 0])
axes[1, 0].set_title('Red Band vs Chla RFU')

# NIR Band
sns.scatterplot(x='nir_standard', y='result_standard', hue='cluster', data=turbid_water, palette='Set1', ax=axes[1, 1])
axes[1, 1].set_title('NIR Band vs Chla RFU')

plt.tight_layout()
plt.show()
'''

"\n# Select the features for clustering (standardized reflectance and chlorophyll-a)\nfeatures_for_clustering = turbid_water[['blue_standard', 'green_standard', 'red_standard', 'nir_standard', 'result_standard']]\n\n# Define the number of clusters (you can adjust this based on your data exploration)\nkmeans = KMeans(n_clusters=3, random_state=42)  # Start with 3 clusters\nturbid_water['cluster'] = kmeans.fit_predict(features_for_clustering)\n\n# Create a scatter plot for each band with clusters\nfig, axes = plt.subplots(2, 2, figsize=(12, 10))\n\n# Blue Band\nsns.scatterplot(x='blue_standard', y='result_standard', hue='cluster', data=turbid_water, palette='Set1', ax=axes[0, 0])\naxes[0, 0].set_title('Blue Band vs Chla RFU')\n\n# Green Band\nsns.scatterplot(x='green_standard', y='result_standard', hue='cluster', data=turbid_water, palette='Set1', ax=axes[0, 1])\naxes[0, 1].set_title('Green Band vs Chla RFU')\n\n# Red Band\nsns.scatterplot(x='red_standard', y='result_standard', hue='clus